# Data download and cleanup

In [1]:
import os
import glob
import pandas as pd

## Download data 

Source: Data were downloaded from the [Medicare public data website](https://data.cms.gov/provider-summary-by-type-of-service/medicare-physician-other-practitioners/medicare-physician-other-practitioners-by-provider) for years 2018, 2019, 2020.

In [2]:
os.getcwd()

'/Users/oana/Documents/Stanford/Spring2023/CS230/final_project/cs230_project/code/preprocessing'

In [3]:
# assuming you're in code/preprocessing 
data_path = '../../data/raw/'
all_data_files = glob.glob(os.path.join(data_path, 'medicare_*.csv'))

data_list = []

for f in all_data_files:
    df = pd.read_csv(f, header = 0, encoding = 'latin-1')
    df['year'] = str.split(str.split(str.split(f, '/')[-1], "_")[-1], ".csv")[0]
    data_list.append(df)

data_df = pd.concat(data_list, axis=0, ignore_index = True)

/var/folders/6m/dcb0chc94pl7hxnxbqp281jc0000gn/T/ipykernel_90325/664181645.py:8: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f, header = 0, encoding = 'latin-1')
/var/folders/6m/dcb0chc94pl7hxnxbqp281jc0000gn/T/ipykernel_90325/664181645.py:8: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f, header = 0, encoding = 'latin-1')


In [4]:
all_data_files #just to check

['../../data/raw/medicare_raw_2019.csv',
 '../../data/raw/medicare_raw_2018.csv']

In [7]:
data_df.shape

(2277332, 74)

In [8]:
for c in data_df.columns:
    print(c)

Rndrng_NPI
Rndrng_Prvdr_Last_Org_Name
Rndrng_Prvdr_First_Name
Rndrng_Prvdr_MI
Rndrng_Prvdr_Crdntls
Rndrng_Prvdr_Gndr
Rndrng_Prvdr_Ent_Cd
Rndrng_Prvdr_St1
Rndrng_Prvdr_St2
Rndrng_Prvdr_City
Rndrng_Prvdr_State_Abrvtn
Rndrng_Prvdr_State_FIPS
Rndrng_Prvdr_Zip5
Rndrng_Prvdr_RUCA
Rndrng_Prvdr_RUCA_Desc
Rndrng_Prvdr_Cntry
Rndrng_Prvdr_Type
Rndrng_Prvdr_Mdcr_Prtcptg_Ind
Tot_HCPCS_Cds
Tot_Benes
Tot_Srvcs
Tot_Sbmtd_Chrg
Tot_Mdcr_Alowd_Amt
Tot_Mdcr_Pymt_Amt
Tot_Mdcr_Stdzd_Amt
Drug_Sprsn_Ind
Drug_Tot_HCPCS_Cds
Drug_Tot_Benes
Drug_Tot_Srvcs
Drug_Sbmtd_Chrg
Drug_Mdcr_Alowd_Amt
Drug_Mdcr_Pymt_Amt
Drug_Mdcr_Stdzd_Amt
Med_Sprsn_Ind
Med_Tot_HCPCS_Cds
Med_Tot_Benes
Med_Tot_Srvcs
Med_Sbmtd_Chrg
Med_Mdcr_Alowd_Amt
Med_Mdcr_Pymt_Amt
Med_Mdcr_Stdzd_Amt
Bene_Avg_Age
Bene_Age_LT_65_Cnt
Bene_Age_65_74_Cnt
Bene_Age_75_84_Cnt
Bene_Age_GT_84_Cnt
Bene_Feml_Cnt
Bene_Male_Cnt
Bene_Race_Wht_Cnt
Bene_Race_Black_Cnt
Bene_Race_API_Cnt
Bene_Race_Hspnc_Cnt
Bene_Race_NatInd_Cnt
Bene_Race_Othr_Cnt
Bene_Dual_Cnt
Bene_Ndual_Cn

## Keep billing-related amounts only 

Used data dictionary [here](https://data.cms.gov/resources/medicare-physician-other-practitioners-by-provider-data-dictionary). 

Assume fraud is largely defined by billing practices, but also included some demographic info as available.

Kept values for drug/prescribing behavior & medical practices separately. 

In [23]:
cols_to_keep = ["Rndrng_NPI",
                "Rndrng_Prvdr_Type",
                'Rndrng_Prvdr_Ent_Cd', # for cleaning
                "Rndrng_Prvdr_Mdcr_Prtcptg_Ind", # for cleaning
                "Rndrng_Prvdr_Gndr",
                "Rndrng_Prvdr_Cntry", # for cleaning 
                "Drug_Sprsn_Ind",
                "Drug_Tot_HCPCS_Cds",
                "Drug_Tot_Benes",
                "Drug_Tot_Srvcs",
                "Drug_Sbmtd_Chrg",
                "Drug_Mdcr_Alowd_Amt",
                "Drug_Mdcr_Pymt_Amt",
                "Drug_Mdcr_Stdzd_Amt", 
                "Med_Sprsn_Ind", 
                "Med_Tot_HCPCS_Cds", 
                "Med_Tot_Benes", 
                "Med_Tot_Srvcs", 
                "Med_Sbmtd_Chrg", 
                "Med_Mdcr_Alowd_Amt", 
                "Med_Mdcr_Pymt_Amt", 
                "Med_Mdcr_Stdzd_Amt"]

In [24]:
data_df2 = data_df[cols_to_keep]

In [25]:
data_df2.shape

(2277332, 22)

## Remove irrelevant columns

In [26]:
# Keep individuals only, denoted by 'I' 
data_df2 = data_df2.loc[data_df2['Rndrng_Prvdr_Ent_Cd'] == 'I']
data_df2.shape

(2156627, 22)

In [27]:
# US data only
data_df2 = data_df2.loc[data_df2['Rndrng_Prvdr_Cntry'] == 'US']
data_df2.shape

(2156287, 22)

In [28]:
# Only providers participating in Medicare
data_df2 = data_df2.loc[data_df2['Rndrng_Prvdr_Mdcr_Prtcptg_Ind'] == 'Y']
data_df2.shape

(2154394, 22)

In [29]:
# Remove data that was supressed already (Drug-related)
data_df2 = data_df2.loc[data_df2['Drug_Sprsn_Ind'].isnull()]
data_df2.shape

(1915045, 22)

In [31]:
# Remove data that was supressed already (Medical-related)
data_df2 = data_df2.loc[data_df2['Med_Sprsn_Ind'].isnull()]
data_df2.shape

(1915045, 22)

In [33]:
# Remove columns used for cleaning only 
data_df3 = data_df2.drop(['Rndrng_Prvdr_Ent_Cd',
                         'Rndrng_Prvdr_Cntry',
                         'Rndrng_Prvdr_Mdcr_Prtcptg_Ind',
                         'Drug_Sprsn_Ind',
                         'Med_Sprsn_Ind'], axis=1)

In [34]:
data_df3.shape

(1915045, 17)

## Fix dtypes

In [66]:
#data_df = data_df.convert_dtypes()

In [36]:
# Turn gender into a binary column 
data_df3['Rndrng_Prvdr_Gndr'] = data_df3['Rndrng_Prvdr_Gndr'].astype('category').cat.codes

In [37]:
# one hot encode categorical columns
data_df3 = pd.get_dummies(data_df3, columns = ['Rndrng_Prvdr_Type'], dtype='int')

In [39]:
data_df3.shape

(1915045, 107)

In [41]:
data_df3.head()

,Rndrng_NPI,Rndrng_Prvdr_Gndr,Drug_Tot_HCPCS_Cds,Drug_Tot_Benes,Drug_Tot_Srvcs,Drug_Sbmtd_Chrg,Drug_Mdcr_Alowd_Amt,Drug_Mdcr_Pymt_Amt,Drug_Mdcr_Stdzd_Amt,Med_Tot_HCPCS_Cds,...,Rndrng_Prvdr_Type_Slide Preparation Facility,Rndrng_Prvdr_Type_Speech Language Pathologist,Rndrng_Prvdr_Type_Sports Medicine,Rndrng_Prvdr_Type_Surgical Oncology,Rndrng_Prvdr_Type_Thoracic Surgery,Rndrng_Prvdr_Type_Undefined Physician type,Rndrng_Prvdr_Type_Undersea and Hyperbaric Medicine,Rndrng_Prvdr_Type_Unknown Supplier/Provider Specialty,Rndrng_Prvdr_Type_Urology,Rndrng_Prvdr_Type_Vascular Surgery
0,1003000126,1,0.0,0.0,0.0,0.00,0.00,0.00,0.00,18.0,...,0,0,0,0,0,0,0,0,0,0
1,1003000134,1,0.0,0.0,0.0,0.00,0.00,0.00,0.00,20.0,...,0,0,0,0,0,0,0,0,0,0
4,1003000480,1,0.0,0.0,0.0,0.00,0.00,0.00,0.00,30.0,...,0,0,0,0,0,0,0,0,0,0
5,1003000522,1,8.0,122.0,140.0,36767.44,12144.39,12121.71,12233.86,38.0,...,0,0,0,0,0,0,0,0,0,0
6,1003000530,0,7.0,130.0,156.0,16992.00,11814.19,11811.69,11811.82,26.0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
data_df4 = data_df3.dropna()

In [43]:
data_df4.shape

(1915045, 107)

## Add labels 

The LEIE database was downloaded from [here](https://oig.hhs.gov/exclusions/exclusions_list.asp) (5/10/2023 update). This is updated montly and indicates providers who are excluded from federally funded HC programs for various reasons. In line with prior work, use indicators most likely to be associated with Medicare fraud [source](https://journalofbigdata.springeropen.com/articles/10.1186/s40537-018-0138-3).

In [44]:
df_leie = pd.read_csv('../../data/raw/leie_5_10_2023.csv', low_memory=False)

In [45]:
# Drop irrelevant columns 
df_leie = df_leie.loc[:,['NPI','EXCLTYPE','EXCLDATE', 'WAIVERDATE']]

#drop NPI = 0
df_leie = df_leie[df_leie["NPI"]>0]

In [46]:
# Remove whitespace and then only keep relevant exclusion codes 
df_leie['EXCLTYPE'] = df_leie['EXCLTYPE'].str.strip()
df_leie = df_leie[df_leie['EXCLTYPE'].isin(['1128a1','1128a2','1128a3','1128b4','1128b7'])]

In [47]:
df_leie.shape

(5461, 4)

In [48]:
df_leie['exclusion_year'] = [x[0:4] for x in df_leie['EXCLDATE'].astype(str)]
df_leie['exclusion_year'] = df_leie['exclusion_year'].astype(int)

In [49]:
df_leie.head()

,NPI,EXCLTYPE,EXCLDATE,WAIVERDATE,exclusion_year
6,1922348218,1128a1,20180419,0,2018
30,1275600959,1128a1,20130320,0,2013
43,1265830335,1128a1,20220818,0,2022
63,1851631543,1128b7,20190326,0,2019
64,1902198435,1128a1,20160120,0,2016


In [50]:
len(df_leie['NPI'].unique())

5389

In [51]:
len(df_leie['NPI'][df_leie['exclusion_year'] >= 2020].unique())

1468

In [52]:
df_leie_after_2019 = df_leie.loc[df_leie['exclusion_year'] >= 2020]

In [53]:
len(df_leie_after_2019['NPI'].unique())

1468

In [54]:
df_leie_after_2019['excluded'] = [1] * df_leie_after_2019.shape[0]

/var/folders/6m/dcb0chc94pl7hxnxbqp281jc0000gn/T/ipykernel_90325/2206586674.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_leie_after_2019['excluded'] = [1] * df_leie_after_2019.shape[0]


In [56]:
df_leie_after_2019.head()

,NPI,EXCLTYPE,EXCLDATE,WAIVERDATE,exclusion_year,excluded
43,1265830335,1128a1,20220818,0,2022,1
65,1073916631,1128b7,20210816,0,2021,1
80,1437510278,1128a1,20230420,0,2023,1
119,1801231436,1128a1,20211029,0,2021,1
139,1750442018,1128a1,20201020,0,2020,1


## Group values by provider 

Since we're trying to flag providers who are fraudulent, we assume they will be fraudulent on average across previouus years. 

In [61]:
data_df_grouped = data_df4.groupby("Rndrng_NPI").mean()

In [62]:
data_df_grouped = data_df_grouped.reset_index()

In [63]:
data_df_grouped.head()

,Rndrng_NPI,Rndrng_Prvdr_Gndr,Drug_Tot_HCPCS_Cds,Drug_Tot_Benes,Drug_Tot_Srvcs,Drug_Sbmtd_Chrg,Drug_Mdcr_Alowd_Amt,Drug_Mdcr_Pymt_Amt,Drug_Mdcr_Stdzd_Amt,Med_Tot_HCPCS_Cds,...,Rndrng_Prvdr_Type_Slide Preparation Facility,Rndrng_Prvdr_Type_Speech Language Pathologist,Rndrng_Prvdr_Type_Sports Medicine,Rndrng_Prvdr_Type_Surgical Oncology,Rndrng_Prvdr_Type_Thoracic Surgery,Rndrng_Prvdr_Type_Undefined Physician type,Rndrng_Prvdr_Type_Undersea and Hyperbaric Medicine,Rndrng_Prvdr_Type_Unknown Supplier/Provider Specialty,Rndrng_Prvdr_Type_Urology,Rndrng_Prvdr_Type_Vascular Surgery
0,1003000126,1.0,0.0,0.0,0.0,0.000,0.000,0.0,0.00,18.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1003000134,1.0,0.0,0.0,0.0,0.000,0.000,0.0,0.00,20.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1003000407,1.0,0.0,0.0,0.0,0.000,0.000,0.0,0.00,23.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1003000480,1.0,0.0,0.0,0.0,0.000,0.000,0.0,0.00,30.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1003000522,1.0,9.0,144.5,167.5,38892.565,13229.245,13197.3,13262.99,40.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [64]:
data_df_grouped.shape

(1079490, 107)

## Combine

In [65]:
combined_df = pd.merge(data_df_grouped, 
                       df_leie_after_2019.loc[:,['NPI', 'excluded']], 
                       left_on='Rndrng_NPI', 
                       right_on='NPI', 
                       how='left')
combined_df = combined_df.drop(['NPI'], axis=1)

In [66]:
combined_df.shape

(1079491, 108)

In [67]:
# Proportion of positive cases
num_fraudulent_providers = len(combined_df['Rndrng_NPI'][combined_df['excluded'] == 1].unique())
num_fraudulent_providers/len(combined_df['Rndrng_NPI'].unique())

0.0003047735504729085

In [68]:
num_fraudulent_providers

329

In [69]:
num_providers = len(combined_df['Rndrng_NPI'].unique())
num_providers

1079490

In [70]:
combined_df.head()

,Rndrng_NPI,Rndrng_Prvdr_Gndr,Drug_Tot_HCPCS_Cds,Drug_Tot_Benes,Drug_Tot_Srvcs,Drug_Sbmtd_Chrg,Drug_Mdcr_Alowd_Amt,Drug_Mdcr_Pymt_Amt,Drug_Mdcr_Stdzd_Amt,Med_Tot_HCPCS_Cds,...,Rndrng_Prvdr_Type_Speech Language Pathologist,Rndrng_Prvdr_Type_Sports Medicine,Rndrng_Prvdr_Type_Surgical Oncology,Rndrng_Prvdr_Type_Thoracic Surgery,Rndrng_Prvdr_Type_Undefined Physician type,Rndrng_Prvdr_Type_Undersea and Hyperbaric Medicine,Rndrng_Prvdr_Type_Unknown Supplier/Provider Specialty,Rndrng_Prvdr_Type_Urology,Rndrng_Prvdr_Type_Vascular Surgery,excluded
0,1003000126,1.0,0.0,0.0,0.0,0.000,0.000,0.0,0.00,18.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,1003000134,1.0,0.0,0.0,0.0,0.000,0.000,0.0,0.00,20.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,1003000407,1.0,0.0,0.0,0.0,0.000,0.000,0.0,0.00,23.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1003000480,1.0,0.0,0.0,0.0,0.000,0.000,0.0,0.00,30.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,1003000522,1.0,9.0,144.5,167.5,38892.565,13229.245,13197.3,13262.99,40.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [71]:
combined_df['excluded'] = combined_df.excluded.fillna(0)

In [72]:
combined_df.head()

,Rndrng_NPI,Rndrng_Prvdr_Gndr,Drug_Tot_HCPCS_Cds,Drug_Tot_Benes,Drug_Tot_Srvcs,Drug_Sbmtd_Chrg,Drug_Mdcr_Alowd_Amt,Drug_Mdcr_Pymt_Amt,Drug_Mdcr_Stdzd_Amt,Med_Tot_HCPCS_Cds,...,Rndrng_Prvdr_Type_Speech Language Pathologist,Rndrng_Prvdr_Type_Sports Medicine,Rndrng_Prvdr_Type_Surgical Oncology,Rndrng_Prvdr_Type_Thoracic Surgery,Rndrng_Prvdr_Type_Undefined Physician type,Rndrng_Prvdr_Type_Undersea and Hyperbaric Medicine,Rndrng_Prvdr_Type_Unknown Supplier/Provider Specialty,Rndrng_Prvdr_Type_Urology,Rndrng_Prvdr_Type_Vascular Surgery,excluded
0,1003000126,1.0,0.0,0.0,0.0,0.000,0.000,0.0,0.00,18.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1003000134,1.0,0.0,0.0,0.0,0.000,0.000,0.0,0.00,20.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1003000407,1.0,0.0,0.0,0.0,0.000,0.000,0.0,0.00,23.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1003000480,1.0,0.0,0.0,0.0,0.000,0.000,0.0,0.00,30.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1003000522,1.0,9.0,144.5,167.5,38892.565,13229.245,13197.3,13262.99,40.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [73]:
combined_df.groupby(['excluded']).size()

excluded
0.0    1079161
1.0        330
dtype: int64

In [74]:
combined_df.to_csv('../../data/processed/combined_processed_data.csv', 
                   index=False)